In [9]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os

def calculate_angle_2d(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360.0 - angle
        
    return angle

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def process_video(file_path, class_name, df):
    cap = cv2.VideoCapture(file_path)

    pose = mp_pose.Pose()
    frame_data = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            try:
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                           landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
                       landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]



                right_shoulder_angle = calculate_angle_2d(right_hip, right_shoulder, right_elbow)
                right_elbow_angle = calculate_angle_2d(right_shoulder, right_elbow, right_wrist)
                right_hip_angle = calculate_angle_2d(right_knee, right_hip, right_shoulder)
                right_knee_angle = calculate_angle_2d(right_hip, right_knee, right_ankle)
                                
                
                left_shoulder_angle = calculate_angle_2d(left_hip, left_shoulder, left_elbow)
                left_elbow_angle = calculate_angle_2d(left_shoulder, left_elbow, left_wrist)
                left_hip_angle = calculate_angle_2d(left_knee, left_hip, left_shoulder)
                left_knee_angle = calculate_angle_2d(left_hip, left_knee, left_ankle)
                
            except IndexError:
                right_shoulder_angle = np.nan
                right_elbow_angle = np.nan
                right_hip_angle = np.nan
                right_knee_angle = np.nan
                left_shoulder_angle = np.nan
                left_elbow_angle = np.nan
                left_hip_angle = np.nan
                left_knee_angle = np.nan
        else:
            right_shoulder_angle = np.nan
            right_elbow_angle = np.nan
            right_hip_angle = np.nan
            right_knee_angle = np.nan
            left_shoulder_angle = np.nan
            left_elbow_angle = np.nan
            left_hip_angle = np.nan
            left_knee_angle = np.nan

        frame_data.append({
            'frame': cap.get(cv2.CAP_PROP_POS_FRAMES),
            'right_shoulder_angle' : right_shoulder_angle,
            'right_elbow_angle' : right_elbow_angle,
            'right_hip_angle' : right_hip_angle,
            'right_knee_angle' : right_knee_angle,
            'left_shoulder_angle' : left_shoulder_angle,
            'left_elbow_angle' : left_elbow_angle,
            'left_hip_angle' : left_hip_angle,
            'left_knee_angle' : left_knee_angle,
            'class_name': class_name
        })

    cap.release()
    df = pd.concat([df, pd.DataFrame(frame_data)], ignore_index=True)
    return df

if __name__ == "__main__":
    video_folder = 'vid'  # Folder containing the videos
    output_folder = 'csv'  # Folder to save CSV files

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Initialize a single DataFrame to hold all data
    all_data = pd.DataFrame()

    # Process each video
    for file_name in os.listdir(video_folder):
        if file_name.endswith('.mp4'):
            class_name = file_name.split('_')[0]  # Extract class name from file name
            file_path = os.path.join(video_folder, file_name)

            # Process the video and append data to the main DataFrame
            all_data = process_video(file_path, class_name, all_data)

    # Save the consolidated DataFrame to a single CSV file
    output_file = os.path.join(output_folder, 'output.csv')
    all_data.to_csv(output_file, index=False)
    print(f'Saved all angles to {output_file}')


C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Saved all angles to csv\output.csv


In [25]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os

def calculate_angle_2d(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360.0 - angle
        
    return angle

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def process_video(file_path, class_name, df, desired_fps=60):
    cap = cv2.VideoCapture(file_path)
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = max(1, int(original_fps / desired_fps))  # Ensure frame_interval is at least 1
    
    pose = mp_pose.Pose()
    frame_data = []

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)

            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark

                try:
                    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
                                landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
                                landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
                                landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, 
                                landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                    
                    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                    right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                    right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

                    right_shoulder_angle = calculate_angle_2d(right_hip, right_shoulder, right_elbow)
                    right_elbow_angle = calculate_angle_2d(right_shoulder, right_elbow, right_wrist)
                    right_hip_angle = calculate_angle_2d(right_knee, right_hip, right_shoulder)
                    right_knee_angle = calculate_angle_2d(right_hip, right_knee, right_ankle)
                                    
                    left_shoulder_angle = calculate_angle_2d(left_hip, left_shoulder, left_elbow)
                    left_elbow_angle = calculate_angle_2d(left_shoulder, left_elbow, left_wrist)
                    left_hip_angle = calculate_angle_2d(left_knee, left_hip, left_shoulder)
                    left_knee_angle = calculate_angle_2d(left_hip, left_knee, left_ankle)
                    
                except IndexError:
                    right_shoulder_angle = np.nan
                    right_elbow_angle = np.nan
                    right_hip_angle = np.nan
                    right_knee_angle = np.nan
                    left_shoulder_angle = np.nan
                    left_elbow_angle = np.nan
                    left_hip_angle = np.nan
                    left_knee_angle = np.nan
            else:
                right_shoulder_angle = np.nan
                right_elbow_angle = np.nan
                right_hip_angle = np.nan
                right_knee_angle = np.nan
                left_shoulder_angle = np.nan
                left_elbow_angle = np.nan
                left_hip_angle = np.nan
                left_knee_angle = np.nan

            frame_data.append({
                'frame': cap.get(cv2.CAP_PROP_POS_FRAMES),
                'right_shoulder_angle' : right_shoulder_angle,
                'right_elbow_angle' : right_elbow_angle,
                'right_hip_angle' : right_hip_angle,
                'right_knee_angle' : right_knee_angle,
                'left_shoulder_angle' : left_shoulder_angle,
                'left_elbow_angle' : left_elbow_angle,
                'left_hip_angle' : left_hip_angle,
                'left_knee_angle' : left_knee_angle,
                'class_name': class_name
            })
        frame_count += 1

    cap.release()
    df = pd.concat([df, pd.DataFrame(frame_data)], ignore_index=True)
    return df

if __name__ == "__main__":
    video_folder = 'vid'  # Folder containing the videos
    output_folder = 'csv'  # Folder to save CSV files

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Initialize a single DataFrame to hold all data
    all_data = pd.DataFrame()

    # Process each video
    for file_name in os.listdir(video_folder):
        if file_name.endswith('.mp4', '.MOV'):
            class_name = file_name.split('_')[0]  # Extract class name from file name
            file_path = os.path.join(video_folder, file_name)

            # Process the video and append data to the main DataFrame
            all_data = process_video(file_path, class_name, all_data)

    # Save the consolidated DataFrame to a single CSV file
    output_file = os.path.join(output_folder, 'output.csv')
    all_data.to_csv(output_file, index=False)
    print(f'Saved all angles to {output_file}')


C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Saved all angles to csv\output.csv


In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Read the CSV file
data = pd.read_csv('csv/output.csv')

# Drop rows with NaN values
#data.dropna(inplace=True)

# Encode the class labels
label_encoder = LabelEncoder()
data['class_name'] = label_encoder.fit_transform(data['class_name'])

# Extract features and labels
features = data[['right_shoulder_angle',
            'right_elbow_angle',
            'right_hip_angle',
            'right_knee_angle',
            'left_shoulder_angle',
            'left_elbow_angle',
            'left_hip_angle',
            'left_knee_angle']].values

labels = data['class_name'].values

# Standardize the features
scaler = StandardScaler()
features = scaler.fit_transform(features)

# Define sequence length
sequence_length = 40

# Prepare sequences for LSTM
def create_sequences(data, labels, seq_length):
    sequences = []
    labels_seq = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        label_seq = labels[i + seq_length - 1]
        sequences.append(seq)
        labels_seq.append(label_seq)
    return np.array(sequences), np.array(labels_seq)

X, y = create_sequences(features, labels, sequence_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels to categorical format
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Build the LSTM model
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(sequence_length, X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=12, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Save the model
model.save('lstm_model.h5')
print('Model saved as lstm_model.h5')


Epoch 1/30


C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


379/379 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.8392 - loss: 0.4829 - val_accuracy: 0.9859 - val_loss: 0.0565
Epoch 2/30
379/379 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9886 - loss: 0.0465 - val_accuracy: 0.9930 - val_loss: 0.0279
Epoch 3/30
379/379 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9951 - loss: 0.0253 - val_accuracy: 0.9974 - val_loss: 0.0117
Epoch 4/30
379/379 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9974 - loss: 0.0118 - val_accuracy: 0.9947 - val_loss: 0.0147
Epoch 5/30
379/379 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.9985 - loss: 0.0063 - val_accuracy: 0.9974 - val_loss: 0.0133
Epoch 6/30
379/379 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9985 - loss: 0.0088 - val_accuracy: 0.9965 - val_loss: 0.0114
Epoch 7/30
379/379 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9975 - loss: 0.0091 - val_accuracy: 0.9974 - val_loss: 0.0108
Epoch 8/30
379/379 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9988 - loss: 0.0038 - val_accuracy: 0.999

Test Accuracy: 99.82%
Model saved as lstm_model.h5


In [29]:
from sklearn.preprocessing import LabelEncoder
import joblib

# Saving the LabelEncoder after training
joblib.dump(label_encoder, 'label_encoder.pkl')


['label_encoder.pkl']